# Imports

In [74]:
import os

import pandas as pd
import matplotlib.pyplot as plt

from macros import column_groups, DATA_PATH, DATA_SPLIT_PATH, ORIGINAL_DATASET_NAME

# Loading

In [76]:
df = pd.read_csv(
    os.path.join(DATA_SPLIT_PATH, "student_questionnaire.csv"), low_memory=False
)
df = df.set_index("id_student")
df

,a1,a2,a3a,a3b,living_with_father_mother,a3c,a3d,a3et,a3f,a4,...,a163k,a166f,a166k,a171h,a177d,a211a,a222b,country_iso_cnac,country_iso_nac,weight
id_student,,,,,,,,,,,,,,,,,,,,,
1,2.0,2007.0,1.0,1.0,1.0,NaN,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,2007.0,1.0,1.0,1.0,1.0,2.0,2.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,2007.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,2007.0,1.0,1.0,1.0,1.0,2.0,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.0,2007.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83853,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,1.0,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN
83854,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,1.0,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN
83855,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,2.0,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN


In [77]:
# Load identifiers and change float columns to int
ids = pd.read_csv(os.path.join(DATA_SPLIT_PATH, "identifiers.csv"), low_memory=False)
ids = ids.set_index("id_student")
int_identifiers = [col for col in ids.columns if col not in ["id_class_group"]]
ids[int_identifiers] = ids[int_identifiers].astype("Int64")
ids

,id_student_original,id_year,id_grade,id_class_group,id_school,id_student_16_19,id_school_16_19
id_student,,,,,,,
1,33613,2016,3,A,2415,14374,569
2,19294,2016,3,A,1842,8001,273
3,19587,2016,3,NaN,1432,8142,82
4,29985,2016,3,A,2280,12800,505
5,6982,2016,3,A,2040,2606,390
...,...,...,...,...,...,...,...
83853,2500,2019,6,B,1278,<NA>,271
83854,5784,2019,6,A,1079,<NA>,78
83855,7708,2019,6,X,1486,3036,459


In [78]:
# Load identifiers and change float columns to int
scores = pd.read_csv(
    os.path.join(DATA_SPLIT_PATH, "student_scores.csv"), low_memory=False
)
scores = scores.set_index("id_student")
# int_identifiers = [col for col in ids.columns if col not in ["id_class_group"]]
# ids[int_identifiers] = ids[int_identifiers].astype('Int64')
scores

,score_MAT,level_MAT,score_LEN,level_LEN,score_ING,level_ING
id_student,,,,,,
1,564.8700,3.0,535.1500,3.0,NaN,NaN
2,388.3400,1.0,293.7000,1.0,NaN,NaN
3,386.5900,1.0,514.8100,3.0,NaN,NaN
4,487.7600,2.0,449.2500,2.0,NaN,NaN
5,709.7900,4.0,598.7200,3.0,NaN,NaN
...,...,...,...,...,...,...
83853,400.8625,2.0,446.6522,2.0,294.7474,1.0
83854,597.0243,3.0,632.6043,4.0,633.2966,4.0
83855,707.9254,4.0,400.2761,2.0,477.5056,2.0


In [79]:
ids = pd.merge(ids, scores, left_index=True, right_index=True)
df = pd.merge(ids, df, left_index=True, right_index=True)
df

,id_student_original,id_year,id_grade,id_class_group,id_school,id_student_16_19,id_school_16_19,score_MAT,level_MAT,score_LEN,...,a163k,a166f,a166k,a171h,a177d,a211a,a222b,country_iso_cnac,country_iso_nac,weight
id_student,,,,,,,,,,,,,,,,,,,,,
1,33613,2016,3,A,2415,14374,569,564.8700,3.0,535.1500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19294,2016,3,A,1842,8001,273,388.3400,1.0,293.7000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19587,2016,3,NaN,1432,8142,82,386.5900,1.0,514.8100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,29985,2016,3,A,2280,12800,505,487.7600,2.0,449.2500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6982,2016,3,A,2040,2606,390,709.7900,4.0,598.7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83853,2500,2019,6,B,1278,<NA>,271,400.8625,2.0,446.6522,...,1.0,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN
83854,5784,2019,6,A,1079,<NA>,78,597.0243,3.0,632.6043,...,1.0,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN
83855,7708,2019,6,X,1486,3036,459,707.9254,4.0,400.2761,...,2.0,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN


# Analysis

## Utils

In [80]:
def print_statistics(columns):
    print("MISSING VALUES")
    # print(df[columns].isna().sum())
    # print()
    # print("Percentage")
    print(df[columns].isna().sum() / df.shape[0] * 100)
    # print()
    # print("DISTINCT VALUES")
    # print(df[columns].value_counts())
    # print()

    corr = df[
        columns
        + [
            "score_MAT",
            "level_MAT",
            "score_LEN",
            "level_LEN",
            "score_ING",
            "level_ING",
        ]
    ].corr()
    
    return corr

## Sex, Birth year

In [81]:
current_columns = ["a1", "a2"]
corr = print_statistics(columns=current_columns)
corr.style.background_gradient(cmap="coolwarm")

MISSING VALUES
a1     4.399156
a2    34.160535
dtype: float64


,a1,a2,score_MAT,level_MAT,score_LEN,level_LEN,score_ING,level_ING
a1,1.000000,0.007121,-0.034178,-0.033343,0.149716,0.144840,0.107751,0.101525
a2,0.007121,1.000000,0.049557,0.048313,0.052564,0.052440,0.060428,0.052448
score_MAT,-0.034178,0.049557,1.000000,0.949125,0.478365,0.458407,0.457797,0.435502
level_MAT,-0.033343,0.048313,0.949125,1.000000,0.456791,0.439989,0.435458,0.416475
score_LEN,0.149716,0.052564,0.478365,0.456791,1.000000,0.949850,0.503339,0.482328
level_LEN,0.144840,0.052440,0.458407,0.439989,0.949850,1.000000,0.481747,0.464060
score_ING,0.107751,0.060428,0.457797,0.435458,0.503339,0.481747,1.000000,0.955457
level_ING,0.101525,0.052448,0.435502,0.416475,0.482328,0.464060,0.955457,1.000000


## Living with

In [86]:
current_columns = ["a3a", "a3b", "living_with_father_mother", "a3c", "a3d", "a3et"]
corr = print_statistics(columns=current_columns)
corr.style.background_gradient(cmap="coolwarm")

MISSING VALUES
a3a                          39.439761
a3b                          44.859702
living_with_father_mother    45.550163
a3c                          49.691737
a3d                          65.330265
a3et                         68.682400
dtype: float64


,a3a,a3b,living_with_father_mother,a3c,a3d,a3et,score_MAT,level_MAT,score_LEN,level_LEN,score_ING,level_ING
a3a,1.000000,0.116596,0.363517,0.166632,-0.094964,-0.061045,-0.036104,-0.036478,-0.039652,-0.039338,-0.065230,-0.060540
a3b,0.116596,1.000000,0.944184,0.276950,-0.107926,-0.131156,-0.063902,-0.062463,-0.066654,-0.060769,-0.085209,-0.076969
living_with_father_mother,0.363517,0.944184,1.000000,0.304234,-0.110912,-0.146009,-0.071954,-0.070433,-0.076143,-0.070631,-0.102154,-0.093072
a3c,0.166632,0.276950,0.304234,1.000000,0.025544,0.029994,-0.012337,-0.011069,-0.001275,-0.001119,-0.011578,-0.004708
a3d,-0.094964,-0.107926,-0.110912,0.025544,1.000000,0.266217,0.076768,0.071484,0.083331,0.081382,0.088625,0.093330
a3et,-0.061045,-0.131156,-0.146009,0.029994,0.266217,1.000000,0.052785,0.050928,0.056121,0.055284,0.055049,0.054594
score_MAT,-0.036104,-0.063902,-0.071954,-0.012337,0.076768,0.052785,1.000000,0.949125,0.478365,0.458407,0.457797,0.435502
level_MAT,-0.036478,-0.062463,-0.070433,-0.011069,0.071484,0.050928,0.949125,1.000000,0.456791,0.439989,0.435458,0.416475
score_LEN,-0.039652,-0.066654,-0.076143,-0.001275,0.083331,0.056121,0.478365,0.456791,1.000000,0.949850,0.503339,0.482328
level_LEN,-0.039338,-0.060769,-0.070631,-0.001119,0.081382,0.055284,0.458407,0.439989,0.949850,1.000000,0.481747,0.464060


## Reaeat and skip

In [87]:
current_columns = ["a4", "repeater", "a5"]
corr = print_statistics(columns=current_columns)
corr.style.background_gradient(cmap="coolwarm")

MISSING VALUES
a4          19.353185
repeater    19.353185
a5          20.396628
dtype: float64


,a4,repeater,a5,score_MAT,level_MAT,score_LEN,level_LEN,score_ING,level_ING
a4,1.000000,0.938139,-0.110773,-0.187035,-0.179028,-0.211072,-0.196200,-0.247258,-0.230254
repeater,0.938139,1.000000,-0.122954,-0.186890,-0.179074,-0.208438,-0.192910,-0.250749,-0.231987
a5,-0.110773,-0.122954,1.000000,0.135348,0.129067,0.139020,0.132801,0.159124,0.150840
score_MAT,-0.187035,-0.186890,0.135348,1.000000,0.949125,0.478365,0.458407,0.457797,0.435502
level_MAT,-0.179028,-0.179074,0.129067,0.949125,1.000000,0.456791,0.439989,0.435458,0.416475
score_LEN,-0.211072,-0.208438,0.139020,0.478365,0.456791,1.000000,0.949850,0.503339,0.482328
level_LEN,-0.196200,-0.192910,0.132801,0.458407,0.439989,0.949850,1.000000,0.481747,0.464060
score_ING,-0.247258,-0.250749,0.159124,0.457797,0.435458,0.503339,0.481747,1.000000,0.955457
level_ING,-0.230254,-0.231987,0.150840,0.435502,0.416475,0.482328,0.464060,0.955457,1.000000


## Homeworks

In [88]:
current_columns = ["a6nm", "a7"]
corr = print_statistics(columns=current_columns)
corr.style.background_gradient(cmap="coolwarm")

MISSING VALUES
a6nm    62.861777
a7      19.784872
dtype: float64


,a6nm,a7,score_MAT,level_MAT,score_LEN,level_LEN,score_ING,level_ING
a6nm,1.000000,0.070558,0.045842,0.040847,0.034946,0.036581,0.008578,0.009152
a7,0.070558,1.000000,0.069256,0.066814,0.076241,0.072528,0.087214,0.083958
score_MAT,0.045842,0.069256,1.000000,0.949125,0.478365,0.458407,0.457797,0.435502
level_MAT,0.040847,0.066814,0.949125,1.000000,0.456791,0.439989,0.435458,0.416475
score_LEN,0.034946,0.076241,0.478365,0.456791,1.000000,0.949850,0.503339,0.482328
level_LEN,0.036581,0.072528,0.458407,0.439989,0.949850,1.000000,0.481747,0.464060
score_ING,0.008578,0.087214,0.457797,0.435458,0.503339,0.481747,1.000000,0.955457
level_ING,0.009152,0.083958,0.435502,0.416475,0.482328,0.464060,0.955457,1.000000


## Frequency of computer

In [89]:
current_columns = ["a8a", "a8b", "a8c"]
corr = print_statistics(columns=current_columns)
corr.style.background_gradient(cmap="coolwarm")

MISSING VALUES
a8a    13.537331
a8b    15.937847
a8c    20.449098
dtype: float64


,a8a,a8b,a8c,score_MAT,level_MAT,score_LEN,level_LEN,score_ING,level_ING
a8a,1.000000,0.203203,0.287153,0.005277,0.003453,0.021739,0.018950,0.065944,0.064127
a8b,0.203203,1.000000,0.077801,-0.009366,-0.012130,-0.020471,-0.020518,0.015083,0.015363
a8c,0.287153,0.077801,1.000000,-0.071665,-0.069213,-0.067247,-0.066593,-0.068104,-0.063728
score_MAT,0.005277,-0.009366,-0.071665,1.000000,0.949125,0.478365,0.458407,0.457797,0.435502
level_MAT,0.003453,-0.012130,-0.069213,0.949125,1.000000,0.456791,0.439989,0.435458,0.416475
score_LEN,0.021739,-0.020471,-0.067247,0.478365,0.456791,1.000000,0.949850,0.503339,0.482328
level_LEN,0.018950,-0.020518,-0.066593,0.458407,0.439989,0.949850,1.000000,0.481747,0.464060
score_ING,0.065944,0.015083,-0.068104,0.457797,0.435458,0.503339,0.481747,1.000000,0.955457
level_ING,0.064127,0.015363,-0.063728,0.435502,0.416475,0.482328,0.464060,0.955457,1.000000


## Classroom condition

In [90]:
current_columns = ["a09a", "a09b", "a09c", "a09d", "a09e"]
corr = print_statistics(columns=current_columns)
corr.style.background_gradient(cmap="coolwarm")

MISSING VALUES
a09a    92.519408
a09b    92.520601
a09c    92.518216
a09d    92.525371
a09e    92.525371
dtype: float64


,a09a,a09b,a09c,a09d,a09e,score_MAT,level_MAT,score_LEN,level_LEN,score_ING,level_ING
a09a,1.000000,0.577021,-0.342760,0.562799,-0.305292,0.099671,0.093014,0.083245,0.088060,0.095469,0.091298
a09b,0.577021,1.000000,-0.402824,0.519614,-0.310097,0.097034,0.091980,0.055425,0.056183,0.072361,0.065211
a09c,-0.342760,-0.402824,1.000000,-0.318280,0.479302,-0.065852,-0.066414,-0.021569,-0.027480,-0.033306,-0.027963
a09d,0.562799,0.519614,-0.318280,1.000000,-0.304948,0.110686,0.101238,0.081053,0.085368,0.096884,0.094197
a09e,-0.305292,-0.310097,0.479302,-0.304948,1.000000,-0.066023,-0.067074,-0.050239,-0.051079,-0.043519,-0.038265
score_MAT,0.099671,0.097034,-0.065852,0.110686,-0.066023,1.000000,0.949125,0.478365,0.458407,0.457797,0.435502
level_MAT,0.093014,0.091980,-0.066414,0.101238,-0.067074,0.949125,1.000000,0.456791,0.439989,0.435458,0.416475
score_LEN,0.083245,0.055425,-0.021569,0.081053,-0.050239,0.478365,0.456791,1.000000,0.949850,0.503339,0.482328
level_LEN,0.088060,0.056183,-0.027480,0.085368,-0.051079,0.458407,0.439989,0.949850,1.000000,0.481747,0.464060
score_ING,0.095469,0.072361,-0.033306,0.096884,-0.043519,0.457797,0.435458,0.503339,0.481747,1.000000,0.955457


## Frequency I use the internet

In [91]:
current_columns = ["a9a", "a9b", "a9c", "a9d", "a9e", "a9f", "a9g"]
corr = print_statistics(columns=current_columns)
corr.style.background_gradient(cmap="coolwarm")

MISSING VALUES
a9a    16.955054
a9b    17.990150
a9c    18.320474
a9d    18.011615
a9e    18.101053
a9f    18.076010
a9g    60.504192
dtype: float64


,a9a,a9b,a9c,a9d,a9e,a9f,a9g,score_MAT,level_MAT,score_LEN,level_LEN,score_ING,level_ING
a9a,1.000000,0.098545,0.174382,0.255882,0.295241,0.235196,0.122068,0.017951,0.016921,0.041534,0.035506,0.064842,0.063517
a9b,0.098545,1.000000,0.264026,0.291024,0.256590,0.149346,0.063746,0.009036,0.008623,-0.051856,-0.052595,-0.011234,-0.009785
a9c,0.174382,0.264026,1.000000,0.214973,0.208912,0.183447,0.108461,-0.025146,-0.024948,-0.066764,-0.064409,-0.055404,-0.051276
a9d,0.255882,0.291024,0.214973,1.000000,0.378066,0.390542,0.375365,0.000147,0.001230,0.026744,0.019762,0.037073,0.038236
a9e,0.295241,0.256590,0.208912,0.378066,1.000000,0.298381,0.194725,0.034553,0.031814,0.044736,0.038923,0.054027,0.052203
a9f,0.235196,0.149346,0.183447,0.390542,0.298381,1.000000,0.605050,-0.022101,-0.021943,-0.002008,-0.005871,-0.017192,-0.016077
a9g,0.122068,0.063746,0.108461,0.375365,0.194725,0.605050,1.000000,-0.078053,-0.075545,-0.021707,-0.024783,-0.041401,-0.041317
score_MAT,0.017951,0.009036,-0.025146,0.000147,0.034553,-0.022101,-0.078053,1.000000,0.949125,0.478365,0.458407,0.457797,0.435502
level_MAT,0.016921,0.008623,-0.024948,0.001230,0.031814,-0.021943,-0.075545,0.949125,1.000000,0.456791,0.439989,0.435458,0.416475
score_LEN,0.041534,-0.051856,-0.066764,0.026744,0.044736,-0.002008,-0.021707,0.478365,0.456791,1.000000,0.949850,0.503339,0.482328


## Work with teachers

In [96]:
current_columns = ["a10a", "a10b", "a10c", "a10d", "a10e", "a10f", "a10g", "a10h", "a10i", "a10j", "a10k", "a10l", "a10m", "a10n"]
corr = print_statistics(columns=current_columns)
corr.style.background_gradient(cmap="coolwarm")

MISSING VALUES
a10a    10.043288
a10b    10.368842
a10c    10.467820
a10d    10.192351
a10e    17.761189
a10f    13.759138
a10g    13.965441
a10h    10.635964
a10i    18.059315
a10j    14.099002
a10k    57.202142
a10l    57.011341
a10m    89.040867
a10n    89.034905
dtype: float64


,a10a,a10b,a10c,a10d,a10e,a10f,a10g,a10h,a10i,a10j,a10k,a10l,a10m,a10n,score_MAT,level_MAT,score_LEN,level_LEN,score_ING,level_ING
a10a,1.000000,0.176691,0.176068,0.154947,0.109036,0.147756,0.121415,0.071812,0.154549,0.158115,0.088556,0.166526,0.232681,0.253554,-0.034934,-0.033739,-0.027960,-0.024996,-0.039945,-0.038285
a10b,0.176691,1.000000,0.231435,0.134526,0.236505,0.119771,0.044889,0.243466,0.169336,0.128476,0.193598,0.154018,0.155862,0.160151,-0.055977,-0.055051,-0.037052,-0.034338,-0.032196,-0.033439
a10c,0.176068,0.231435,1.000000,0.263285,0.243748,0.171843,0.073668,0.153344,0.144053,0.197936,0.268490,0.201121,0.153114,0.215634,-0.026498,-0.027986,-0.014603,-0.012747,-0.019537,-0.019831
a10d,0.154947,0.134526,0.263285,1.000000,0.187879,0.223180,0.067946,0.112290,0.138238,0.151454,0.168405,0.192747,0.282046,0.292107,0.039939,0.036784,0.052948,0.053886,0.071252,0.065597
a10e,0.109036,0.236505,0.243748,0.187879,1.000000,0.092775,0.043575,0.176343,0.139017,0.108675,0.265417,0.139112,0.095995,0.139537,-0.034537,-0.034631,-0.031875,-0.028469,-0.040455,-0.039059
a10f,0.147756,0.119771,0.171843,0.223180,0.092775,1.000000,0.112128,0.094520,0.091473,0.231252,0.052776,0.199539,0.372863,0.266086,0.079628,0.075521,0.097779,0.096708,0.082850,0.075032
a10g,0.121415,0.044889,0.073668,0.067946,0.043575,0.112128,1.000000,-0.101840,0.087996,0.253658,0.097048,0.057022,0.151839,0.100562,-0.008780,-0.008084,0.004885,0.005740,0.024016,0.021846
a10h,0.071812,0.243466,0.153344,0.112290,0.176343,0.094520,-0.101840,1.000000,0.134295,0.057586,0.168622,0.137647,0.136734,0.159034,-0.038273,-0.038502,-0.027751,-0.027523,-0.028517,-0.028313
a10i,0.154549,0.169336,0.144053,0.138238,0.139017,0.091473,0.087996,0.134295,1.000000,0.146080,0.131464,0.102292,0.327541,0.195208,-0.076632,-0.073068,-0.044338,-0.045678,-0.066438,-0.065993
a10j,0.158115,0.128476,0.197936,0.151454,0.108675,0.231252,0.253658,0.057586,0.146080,1.000000,0.056146,0.060025,0.134972,0.134532,0.006727,0.005101,0.004862,0.006721,0.004079,0.001538


## Materials in class

In [97]:
column_number = 11
column_letters = ["a", "b", "c", "d", "e", "f", "g", "h"]
current_columns = [f"a{column_number}{letter}" for letter in column_letters]
corr = print_statistics(columns=current_columns)
corr.style.background_gradient(cmap="coolwarm")

MISSING VALUES
a11a     9.847717
a11b    10.309217
a11c    10.647889
a11d    10.706321
a11e    10.433238
a11f    25.093910
a11g    57.134169
a11h    10.170886
dtype: float64


,a11a,a11b,a11c,a11d,a11e,a11f,a11g,a11h,score_MAT,level_MAT,score_LEN,level_LEN,score_ING,level_ING
a11a,1.000000,0.113518,0.005493,-0.116072,-0.070896,0.127422,-0.037000,-0.003158,0.004460,0.005183,0.015164,0.014652,0.017601,0.014332
a11b,0.113518,1.000000,0.241375,0.136831,0.146794,0.131554,0.223828,0.125608,-0.055181,-0.052437,-0.043688,-0.041520,-0.045914,-0.045732
a11c,0.005493,0.241375,1.000000,0.138273,0.191292,0.076588,0.204435,0.106336,-0.062688,-0.059335,-0.070776,-0.065948,-0.060075,-0.056722
a11d,-0.116072,0.136831,0.138273,1.000000,0.264102,0.003624,0.174234,0.191609,-0.030318,-0.032362,-0.044907,-0.042734,-0.005584,-0.006026
a11e,-0.070896,0.146794,0.191292,0.264102,1.000000,0.068397,0.223303,0.254113,0.000637,-0.001009,0.010197,0.009676,0.025933,0.023297
a11f,0.127422,0.131554,0.076588,0.003624,0.068397,1.000000,0.133564,0.107500,0.019088,0.017367,0.041279,0.036043,0.031067,0.026252
a11g,-0.037000,0.223828,0.204435,0.174234,0.223303,0.133564,1.000000,0.139241,-0.082776,-0.080695,-0.081311,-0.079082,-0.086593,-0.084554
a11h,-0.003158,0.125608,0.106336,0.191609,0.254113,0.107500,0.139241,1.000000,-0.015514,-0.019788,-0.003398,-0.000368,-0.022076,-0.020845
score_MAT,0.004460,-0.055181,-0.062688,-0.030318,0.000637,0.019088,-0.082776,-0.015514,1.000000,0.949125,0.478365,0.458407,0.457797,0.435502
level_MAT,0.005183,-0.052437,-0.059335,-0.032362,-0.001009,0.017367,-0.080695,-0.019788,0.949125,1.000000,0.456791,0.439989,0.435458,0.416475
